In [1]:
# Code of Last assignment

# Start by installing necessary libraries
!conda install --yes beautifulsoup4 #Installing beautifulsoup
!conda install --yes lxml #Installing Parser
!conda install --yes requests #Installing Requests
!pip install geocoder #Installing Geocoder 


# Importing
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder # import geocoder


# Getting the HTML file
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

# Parsing Column Names
column_names = []
n_rows=0

rows = soup.find('table', class_='wikitable').find_all('tr')

for row in rows:
    n_rows+=1
    th_tags = row.find_all('th')
    for th in th_tags:
        text = th.get_text()
        column_names.append(text.rstrip("\n\r"))

# Creating Data Frame

df = pd.DataFrame(columns = column_names,index= range(0,n_rows))

row_index = 0
for row in rows:
    column_index = 0
    columns = row.find_all('td')
    for column in columns:
        text = column.get_text()
        df.iat[row_index,column_index] = text.rstrip("\n\r") # Cleaning \n
        column_index += 1
    row_index += 1
# Cleaning Data Frame from NaN's and Not Assigned

df.drop([0], axis=0, inplace=True) # Droping first NaN line

df_valuable = df[(df['Borough'] != 'Not assigned')] #Removing Not Assigned Borough

df_clean = df_valuable.where(df_valuable != 'Not assigned',df['Borough'], axis =0) #Copying Neighbourhood

df_clean.head(10)

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [33]:
# Not possible to use Geocoder, so I'm loading the csv file

df_coordinates = pd.read_csv('http://cocl.us/Geospatial_data')

dic_coordinates = df_coordinates.set_index('Postal Code').to_dict() #Creating a dictionary for coordinates

In [40]:
# This For-loop checks the latitude and longitude in the dictionary and passes it to a new dataframe

df_latitude_and_longitude = df_clean

for index, row in df_latitude_and_longitude.iterrows():
    df_latitude_and_longitude.at[index,'Latitude'] = dic_coordinates["Latitude"][row['Postcode']]
    df_latitude_and_longitude.at[index,'Longitude'] = dic_coordinates["Longitude"][row['Postcode']]
df_latitude_and_longitude.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763


{'key_1': 1, 'key_2': 2}
